# 下拉式選單
- 例如 sel_element = Select(driver.find_element(By.CSS_SELECTOR, '選擇器字串'))
  - 透過 option 的內文來選擇
    - sel_element.select_by_visible_text('民國 100 年')
  - 透過 option 的 value 屬性所設定值來選擇
    - sel_element.select_by_value('2')
  - 透過 option 的順序索引 (從 0 開始，類似陣列的索引概念) 來選擇
    - sel_element.select_by_index(8)


# 擷圖
- driver.save_screenshot('/path/圖片存放路徑.png')

In [ ]:
'''
臺灣證券交易所
外資及陸資買賣超彙總表
https://www.twse.com.tw/zh/page/trading/fund/TWT38U.html

目標:
整合下拉式選單與元素的定位與操控，來下載交易資料，並擷圖
'''

'''
匯入套件
'''
# 操作 browser 的 驅動程式
from selenium import webdriver

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制停止/強制等待 (程式執行期間休息一下)
from time import sleep

# 建立資料夾與執行檔案相關操作
import os

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

'''
[1] Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
[2] Selenium with Python
https://selenium-python.readthedocs.io/
[3] selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
[4] How to select a drop-down menu value with Selenium using Python?
參考網址：https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                # 不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         # 最大化視窗
my_options.add_argument("--incognito")               # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")   # 取消通知

# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'files')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
# 自訂下載路徑 (不會詢問下載位置)
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath,
    "profile.default_content_settings.popups": 0,
    "download.prompt_for_download": False,
    # "download.directory_upgrade": True,
    # "safebrowsing_for_trusted_sources_enabled": False,
    # "safebrowsing.enabled": False,
    # "plugins.always_open_pdf_externally": True
})

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 走訪網址
url = 'https://www.twse.com.tw/zh/page/trading/fund/TWT38U.html'

In [2]:
# 走訪頁面
def visit():
    driver.get(url)

# 選取下拉式選單的項目
def setDropDownMenu(year, value, index):
    # 強制等待
    sleep(1)

    # 選擇 select[name="yy"] 元素，並依 option 的 innerText 來進行選取
    yy = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=yy]'))
    yy.select_by_visible_text(f'民國 {year} 年')

    # 選擇 select[name="mm"] 元素，並依 option 的 value 來進行選取
    mm = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=mm]'))
    mm.select_by_value(str(value))

    # 選擇 select[name="dd"] 元素，並依 option 的 index 來進行選取
    dd = Select(driver.find_element(By.CSS_SELECTOR, 'select[name=dd]'))
    dd.select_by_index(index)

    # 按下查詢
    driver.find_element(
        By.CSS_SELECTOR, 
        'div.submit'
    ).click()
    
# 下載檔案
def download(year, value, index):
    # 下載 csv
    year = 1911 + year
    value = '0' + str(value) if value < 10 else str(value)
    index = '0' + str(index + 1) if (index + 1) < 10 else str(index + 1)
    date = f'{year}{value}{index}'
    os.system(f'curl "https://www.twse.com.tw/rwd/zh/fund/TWT38U?date={date}&response=csv" -o {folderPath}/{date}.csv')
    
    # 擷圖
    driver.save_screenshot(f"{folderPath}/{date}.png")

# 關閉瀏覽器
def close():
    driver.quit()

In [ ]:
# 走訪頁面
visit()

# 指定年、月、日，檢視查詢結果，並下載 csv
year = 100
value = 2
index = 8
setDropDownMenu(year, value, index)
download(year, value, index)

In [37]:
# 關閉瀏覽器
close()